In [1]:
import cv2
import numpy as np
import mediapipe as mp
import os

In [2]:
mp_pose = mp.solutions.pose 
pose = mp_pose.Pose( 
    static_image_mode=False,
    model_complexity=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.75) # чем больше значение тем больше задержка, но и лучшая надёжность решения

mp_drawing = mp.solutions.drawing_utils # утилита для отрисовки ключевых точек


In [3]:
# получение значений из results 
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33 * 4)

    return pose 

In [4]:
def pose_detection(frame):
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = pose.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    return image, results

In [16]:
# Тест
cap = cv2.VideoCapture(0)

while cap.isOpened():
        ret, frame = cap.read()

        image, results = pose_detection(frame)

        # Rendering Results
        if results.pose_landmarks:
        # to change the colors you use this mp.drawing
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(86, 45, 250), thickness=2, circle_radius=4), )
                    
        cv2.imshow('Hand Tracking', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

print(extract_keypoints(results))
cap.release()
cv2.destroyAllWindows()

[ 0.32912534  0.87286383 -2.13815451  0.99956828  0.39153999  0.74069947
 -2.11188936  0.99905682  0.42467088  0.7376579  -2.11318445  0.99887073
  0.45052028  0.73547298 -2.11302137  0.9988907   0.29006946  0.73620421
 -2.07211399  0.99946791  0.2633366   0.72864914 -2.07381773  0.99957126
  0.24125774  0.72085595 -2.07514191  0.99961013  0.4900609   0.72536838
 -1.63054276  0.99926418  0.22781464  0.70737451 -1.45321333  0.99929881
  0.4007321   0.93961966 -1.91969264  0.99672747  0.2817885   0.94465661
 -1.8774538   0.99725252  0.71448112  1.05242789 -1.18479621  0.99191028
  0.09111948  1.00932145 -0.92027301  0.99110353  0.84957141  1.56674051
 -1.10043931  0.29313809 -0.01203647  1.53988922 -1.039644    0.2345634
  0.82958281  1.9367615  -1.23577189  0.01989511 -0.01738218  1.75815904
 -1.55761194  0.06288922  0.84419382  2.05574059 -1.31300056  0.02866671
 -0.0337132   1.82875228 -1.68967021  0.08448915  0.81451845  2.02368999
 -1.39780271  0.0394153  -0.01894726  1.78230929 -1.

In [5]:
DATA_PATH = './sequence_data'
NP_DATA_PATH = './np_sequence_data'

actions = ['none', 'down', 'right', 'shake', 'static', 'to_fist', 'up', 'z_write']

# создание папки для датасета
try:
    os.makedirs(NP_DATA_PATH)
except FileExistsError:
    pass

for action in actions:

    # создание папок для actions в np data
    try:
        os.makedirs(os.path.join(NP_DATA_PATH, action))
    except FileExistsError:
        pass

    for subdir in os.listdir(os.path.join(DATA_PATH, action)):
        # создание папок для последовтельности в actions np data
        try:
            os.makedirs(os.path.join(NP_DATA_PATH, action, subdir))
        except FileExistsError:
            pass
        
        file_count = 0
        # Перебираем файлы в исходной вложенной папке
        for filename in os.listdir(os.path.join(DATA_PATH, action, subdir)):
            file_path = os.path.join(DATA_PATH, action, subdir, filename)

            # Загружаем изображение с помощью OpenCV
            image = cv2.imread(file_path)
            image, results = pose_detection(image)

            # Export keypoints
            keypoints = extract_keypoints(results)

            npy_path = os.path.join(NP_DATA_PATH, action, subdir, str(file_count))
            np.save(npy_path, keypoints)

            file_count += 1